In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as k
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
from tensorflow.python.client import device_lib

config = tf.compat.v1.ConfigProto(log_device_placement=True,
      gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9))
      
sess = tf.compat.v1.Session(config=config)
physical_devices = tf.config.list_physical_devices('GPU') 
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  print('failed to set growth')

K.set_image_data_format('channels_first');

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:41:00.0, compute capability: 7.5



In [8]:
data = np.load(r"../TrainingData/data.npz")
x_train = data['n_frames']
print(x_train[0].shape)

(9, 8, 128, 128)


In [11]:
from FeedbackGAN import FeedbackGAN
try: del gan
except: pass
gan = FeedbackGAN(channels=5, do_batch_norm=False,use_noise=True)
gan.compile(loss='binary_crossentropy', g_optimizer = RMSprop(learning_rate=.01), d_optimizer = Adam(learning_rate=.01),run_eagerly=True);
gan.build([50,5,8,128,128]);
gan.summary()

Model: "feedback_gan_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Generator (SimpleUGen)       multiple                  1458491   
_________________________________________________________________
Discriminator (discriminator multiple                  36073     
_________________________________________________________________
concatenate_3 (Concatenate)  multiple                  0         
Total params: 1,494,566
Trainable params: 1,494,564
Non-trainable params: 2
_________________________________________________________________


In [12]:
#from Discriminator import discriminator
#disc = discriminator(channels=5)
#disc.build([50,5,8,128,128]);
#disc.summary()

In [13]:
gan.fit(x_train,x_train,epochs=50,batch_size=15,shuffle=True)

Epoch 1/50
55/55 [==============================] - 71s 1s/step - loss: 1.0921 - generator_tricking_metric: 4692.0000 - discriminator_failing_metric: 458.0000
Epoch 2/50
55/55 [==============================] - 69s 1s/step - loss: 2.9229 - generator_tricking_metric: 936.0000 - discriminator_failing_metric: 290.0000
Epoch 3/50
32/55 [================>.............] - ETA: 28s - loss: 3.6820 - generator_tricking_metric: 2258.0000 - discriminator_failing_metric: 239.0000

KeyboardInterrupt: 

In [6]:
from PIL import Image
vid = x_train[1:2,0:5,...]
print(vid.shape)
for f in range(50):
  frame = gan.Generator.predict(vid[:,f:f+5,...])
  vid = np.append(vid,frame,axis=1)

def save_gif(frames,name:str):
    if name[-4:] != '.gif':
        name += '.gif'
    frames[0].save(name,format='GIF',append_images=frames[1:],save_all=True,loop=0)

save_gif([Image.fromarray(vid[0,j,4,:,:]*255) for j in range(15)],'prediction.gif')

save_gif([Image.fromarray(vid[0,j,4,:,:]*255) for j in range(vid.shape[1])],'super_prediction.gif')

(1, 5, 8, 128, 128)
